In [1]:
from mapper import write_time_based_train_test_split
from mapper import map_to_PGPR

In [2]:
dataset_name = 'lfm'

In [3]:
write_time_based_train_test_split(dataset_name, "pgpr", 0.8)

In [4]:
map_to_PGPR('lfm')

In [1]:
%cd models/PGPR


/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/PGPR


In [6]:
%run preprocess.py --dataset lfm

Load lfm dataset from file...
Load user of size 9455
Load product of size 173788
Load artist of size 14987
Load album of size 45978
Load genre of size 749
Load micro_genre of size 1921
Load in_album of size 111577
Load has_micro_genre of size 667023
Load created_by of size 111577
Load has_genre of size 111577
0 0
Load review of size 3941518 with positive reviews= 0  and negative reviews= 3941518
Create lfm knowledge graph from dataset...
Load entities...
Total 246884 nodes.
Load reviews...
Total 7883036 review edges.
Load knowledge created_by...
Total 223154 created_by edges.
Load knowledge has_micro_genre...
Total 1334046 has_micro_genre edges.
Load knowledge has_genre...
Total 223154 has_genre edges.
Load knowledge in_album...
Total 223154 in_album edges.
Remove duplicates...
Compute node degrees...
Generate lfm train/test labels.


In [2]:
%run train_transe_model --dataset lfm

../../data/lfm/preprocessed/pgpr/tmp
[INFO]  Namespace(dataset='lfm', name='train_transe_model', seed=123, gpu='0', epochs=30, batch_size=64, lr=0.5, weight_decay=0, l2_lambda=0, max_grad_norm=5.0, embed_size=100, num_neg_samples=5, steps_per_checkpoint=200, device=device(type='cuda', index=0), log_dir='../../data/lfm/preprocessed/pgpr/tmp/train_transe_model')
Load embeddings ../../data/lfm/preprocessed/pgpr/tmp/train_transe_model/transe_model_sd_epoch_30.ckpt


In [2]:
%run train_agent --dataset lfm

[INFO]  Namespace(dataset='lfm', name='train_agent', seed=123, gpu='0', epochs=50, batch_size=32, lr=0.0001, max_acts=250, max_path_len=3, gamma=0.99, ent_weight=0.001, act_dropout=0, state_history=1, hidden=[512, 256], device=device(type='cuda', index=0), log_dir='../../data/lfm/preprocessed/pgpr/tmp/train_agent')
Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl
[INFO]  Parameters:['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias', 'actor.weight', 'actor.bias', 'critic.weight', 'critic.bias']


/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/PGPR/train_agent.py:50: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cuda/Indexing.cu:1440.)
  actor_logits[1 - act_mask] = -999999.0
/home/jfallmann/miniconda3/envs/bsc/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cuda/Indexing.cu:1440.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[INFO]  epoch/step=1/100 | loss=0.37647 | ploss=0.26276 | vloss=0.12815 | entropy=-14.43317 | reward=0.16084
[INFO]  epoch/step=1/200 | loss=0.18247 | ploss=0.07980 | vloss=0.11705 | entropy=-14.36958 | reward=0.15693
[INFO]  epoch/step=2/300 | loss=0.26236 | ploss=0.15892 | vloss=0.11780 | entropy=-14.35787 | reward=0.16377
[INFO]  epoch/step=2/400 | loss=0.14128 | ploss=0.04370 | vloss=0.11196 | entropy=-14.37236 | reward=0.16016
[INFO]  epoch/step=2/500 | loss=0.14769 | ploss=0.05588 | vloss=0.10618 | entropy=-14.36755 | reward=0.15888
[INFO]  epoch/step=3/600 | loss=0.16835 | ploss=0.08030 | vloss=0.10241 | entropy=-14.36520 | reward=0.15782
[INFO]  epoch/step=3/700 | loss=0.19628 | ploss=0.10005 | vloss=0.11065 | entropy=-14.42180 | reward=0.16272
[INFO]  epoch/step=3/800 | loss=0.15397 | ploss=0.05865 | vloss=0.10971 | entropy=-14.38560 | reward=0.16330
[INFO]  epoch/step=4/900 | loss=0.16999 | ploss=0.07735 | vloss=0.10706 | entropy=-14.41882 | reward=0.15938
[INFO]  epoch/step=

In [5]:
%run test_agent --dataset lfm

Predicting paths...
Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl



100%|█████████▉| 9440/9455 [6:17:33<00:28,  1.92s/it]
9456it [6:18:42,  2.40s/it]                          


Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl
Normalizing items scores...
Saving pred_paths...
Average test set size:  104.71782125859333
Overall for noOfUser=9455, ndcg=0.0906


Overall for noOfUser=9455, hr=0.1687


Overall for noOfUser=9455, precision=0.0222


Overall for noOfUser=9455, recall=0.0025




In [1]:
import numpy as np
import pandas as pd
from random import seed, randint, choice
from collections import defaultdict
import pickle

In [2]:
dataset_name = "lfm"

In [3]:
models = ["pgpr"]
users_topk = {model: defaultdict(list) for model in models}

In [4]:
train_labels = {}
test_labels = {}
ndcgs = {model: [] for model in models}
recalls = {model: [] for model in models}
precisions = {model: [] for model in models}

In [5]:
def dcg_at_k(topk, k, method=1):
    topk = np.asfarray(topk)[:k]
    if topk.size:
        if method == 0:
            return topk[0] + np.sum(topk[1:] / np.log2(np.arange(2, topk.size + 1)))
        elif method == 1:
            return np.sum(topk / np.log2(np.arange(2, topk.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.

def ndcg_at_k(topk, k, method=1):
    dcg_max = dcg_at_k(sorted(topk, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(topk, k, method) / dcg_max

def recall_at_k(topk, test_pids):
    return sum(topk) / len(test_pids)

def precision_at_k(topk, k):
    return sum(topk) / k

In [6]:
from knowledge_graph_utils import entity2plain_text
entity2name = {}
entity2name["pgpr"] = entity2plain_text('lfm', "pgpr")

In [7]:
def path_len(path):
    len = 0
    for s in path:
        if type(s) != str:
            s = str(s)
        if s.isnumeric():
            len+=1
    return len

# Path Structure: user 5038 watched product 2430 watched user 1498 watched product 1788
def template(curr_model, path):
    if path[0] == "self_loop":
        path = path[1:]

    path_length = path_len(path)
    for i in range(1, len(path)):
        s = str(path[i])
        if s.isnumeric():
            if path[i-1] == 'user': continue
            if int(path[i]) not in entity2name[curr_model][path[i-1]]: continue
            path[i] = entity2name[curr_model][path[i-1]][int(path[i])]
    if path_length == 4:
        _, uid, rel_0, e_type_1, e_1, rel_1, e_type_2, e_2, rel_k, _, pid  = path
        return f"{pid} is recommend to you because you {rel_0} {e_1} also {rel_k} by {e_2}"
    elif path_len(path) == 3:
        _, uid, rel_0, e_type_1, e_1, rel_1, _, pid  = path
        return f"{pid} is recommend to you because is {r_k} with {e_1} that you previously {rel_0}"

In [8]:
curr_model = "pgpr"

In [12]:
with open(f"results/lfm/pgpr/pred_paths.pkl", 'rb') as pred_paths_file:
    pred_paths_pgpr = pickle.load(pred_paths_file)
pred_paths_file.close()

In [13]:
pred_paths_pgpr

[[3629,
  169857,
  '0.4625122842480016',
  0.006300935,
  'self_loop user 3629 listened_to product 168414 listened_to user 1510 listened_to product 169857'],
 [3629,
  169857,
  '0.4625122842480016',
  0.00013643457,
  'self_loop user 3629 listened_to product 168232 listened_to user 6900 listened_to product 169857'],
 [3629,
  169857,
  '0.4625122842480016',
  6.3988395e-05,
  'self_loop user 3629 listened_to product 92409 listened_to user 1510 listened_to product 169857'],
 [3629,
  168801,
  '0.6026047023296848',
  0.00093470066,
  'self_loop user 3629 listened_to product 168414 listened_to user 3640 listened_to product 168801'],
 [3629,
  168801,
  '0.6026047023296848',
  0.00026220403,
  'self_loop user 3629 listened_to product 168414 listened_to user 3046 listened_to product 168801'],
 [3629,
  168801,
  '0.6026047023296848',
  0.0001249351,
  'self_loop user 3629 listened_to product 168414 listened_to user 3115 listened_to product 168801'],
 [3629,
  168801,
  '0.602604702329684

In [20]:
df = pd.DataFrame(pred_paths_pgpr[:50], columns=["uid", "pid", "path_score", "path_probability", "path"])


In [21]:
df

,uid,pid,path_score,path_probability,path
0,3629,169857,0.4625122842480016,6.300935e-03,self_loop user 3629 listened_to product 168414...
1,3629,169857,0.4625122842480016,1.364346e-04,self_loop user 3629 listened_to product 168232...
2,3629,169857,0.4625122842480016,6.398839e-05,self_loop user 3629 listened_to product 92409 ...
3,3629,168801,0.6026047023296848,9.347007e-04,self_loop user 3629 listened_to product 168414...
4,3629,168801,0.6026047023296848,2.622040e-04,self_loop user 3629 listened_to product 168414...
5,3629,168801,0.6026047023296848,1.249351e-04,self_loop user 3629 listened_to product 168414...
6,3629,168801,0.6026047023296848,1.910633e-04,self_loop user 3629 listened_to product 168232...
7,3629,168801,0.6026047023296848,4.671332e-04,self_loop user 3629 listened_to product 59404 ...
8,3629,168801,0.6026047023296848,9.851356e-05,self_loop user 3629 listened_to product 59404 ...
9,3629,168801,0.6026047023296848,2.358400e-08,self_loop user 3629 listened_to product 40275 ...


In [22]:
header = ["uid", "pid", "path_score", "path_prob", "path"]
pred_paths_map_pgpr = defaultdict(dict)
for record in pred_paths_pgpr:
    uid, pid, path_score, path_prob, path = record
    if pid not in pred_paths_map_pgpr[uid]:
        pred_paths_map_pgpr[uid][pid] = []
    pred_paths_map_pgpr[uid][pid].append((float(path_score), float(path_prob), path))

In [23]:
n_users = len(pred_paths_map_pgpr.keys())
random_user = randint(0, n_users)
random_product = choice(list(pred_paths_map_pgpr[random_user].keys()))
print(pred_paths_map_pgpr[random_user][random_product])

[(0.5688556260615155, 0.00022079600603319705, 'self_loop user 5375 listened_to product 4346 listened_to user 2292 listened_to product 171671'), (0.5688556260615155, 5.015231727156788e-05, 'self_loop user 5375 listened_to product 108139 listened_to user 2292 listened_to product 171671'), (0.5688556260615155, 0.0004437823372427374, 'self_loop user 5375 listened_to product 43564 listened_to user 2292 listened_to product 171671')]
